# Initial testing

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import tqdm
import csv

from vmc.interface import *
from vmc import run_all

from IPython.core.pylabtools import figsize
from IPython.core.display import display

In [2]:
sns.set(color_codes=True)

## Simplest case

We look at one particle in one dimensions for the simple Gaussian in the harmonic oscillator potential. We restrict ourselves to $\alpha \in [0.3, 0.7]$ as we know that the true minimum is located at $\alpha_0 = 0.5$. We look at 11 values of $\alpha$ in this range. Using natural units and we will only look at $\omega = 1.0$.

We will in this notebook only look at the brute force Metropolis algorithm. We use a step length of $0.5$. This means that the initial distribution and every proposed step will be in the range $[-0.5, 0.5]$. To get good results we use $2^{21}$ Monte Carlo cycles where we start by thermalizing the system for $10\%$ of the number of Monte Carlo cycles.

In [3]:
mass = 1.0
omega = 1.0

num_particles = 1
num_dimensions = 1

alpha_min = 0.3
alpha_max = 0.7
num_alphas = 11

bootstrap_samples = 0

alphas = np.linspace(alpha_min, alpha_max, num_alphas).reshape(num_alphas, 1)
parameter_names = ["alpha"]

step_length = 0.5

num_samples = int(2**21)
#num_samples = int(1e6)
num_thermalization_steps = int(0.1*num_samples)

kwargs = {
    "num_samples": num_samples, "step_length": step_length,
    "num_thermalization_steps": num_thermalization_steps}

In [4]:
wavefunction_analytical = PySimpleGaussian(
    num_particles, num_dimensions, mass, omega, spread=step_length)
wavefunction_approximate = PySimpleGaussianNumerical(
    num_particles, num_dimensions, mass, omega, spread=step_length)

hamiltonian = PyHarmonicOscillator()

sampler_analytical = PySampler(
    wavefunction_analytical, hamiltonian, PyMetropolisAlgorithm())
sampler_approximate = PySampler(
    wavefunction_approximate, PyHarmonicOscillator(), PyMetropolisAlgorithm())

In [5]:
df_analytical = run_all(
    sampler_analytical, alphas, parameter_names, bootstrap_samples, **kwargs)

100%|██████████| 11/11 [00:04<00:00,  2.36it/s]


In [6]:
df_approximate = run_all(
    sampler_approximate, alphas, parameter_names, bootstrap_samples, **kwargs)

100%|██████████| 11/11 [00:06<00:00,  1.75it/s]


In [7]:
display(df_analytical)

,alpha,energy,variance,std,acceptance,sampling_time,block_var,block_std
0,0.30,0.562444,6.540292e-08,0.000256,0.891855,0.432384,1.664181e-06,0.001290
1,0.34,0.536520,3.644956e-08,0.000191,0.884648,0.383399,8.797696e-07,0.000938
2,0.38,0.518630,1.817946e-08,0.000135,0.878443,0.372002,3.989991e-07,0.000632
3,0.42,0.508048,7.357474e-09,0.000086,0.871605,0.379778,1.456656e-07,0.000382
4,0.46,0.502015,1.688052e-09,0.000041,0.865939,0.372143,3.051892e-08,0.000175
5,0.50,0.500000,0.000000e+00,0.000000,0.860474,0.371616,0.000000e+00,0.000000
6,0.54,0.501829,1.377996e-09,0.000037,0.855427,0.379587,2.196225e-08,0.000148
7,0.58,0.505664,5.262149e-09,0.000073,0.849938,0.377779,8.326318e-08,0.000289
8,0.62,0.511751,1.125217e-08,0.000106,0.844786,0.373682,1.683143e-07,0.000410
9,0.66,0.519065,1.905331e-08,0.000138,0.839997,0.372568,2.672381e-07,0.000517


In [8]:
display(df_approximate)

,alpha,energy,variance,std,acceptance,sampling_time,block_var,block_std
0,0.30,0.566752,6.772798e-08,0.000260,0.890983,0.526336,1.814346e-06,0.001347
1,0.34,0.537138,3.741855e-08,0.000193,0.884786,0.521860,9.247553e-07,0.000962
2,0.38,0.518357,1.819723e-08,0.000135,0.878199,0.523505,3.891369e-07,0.000624
3,0.42,0.507174,7.263032e-09,0.000085,0.871802,0.528921,1.411183e-07,0.000376
4,0.46,0.501844,1.716542e-09,0.000041,0.865806,0.517586,3.059459e-08,0.000175
5,0.50,0.499996,3.769995e-11,0.000006,0.860243,0.524273,1.982098e-10,0.000014
6,0.54,0.501326,1.472062e-09,0.000038,0.855196,0.520520,2.270998e-08,0.000151
7,0.58,0.505197,5.336906e-09,0.000073,0.849710,0.521564,8.094387e-08,0.000285
8,0.62,0.511202,1.129796e-08,0.000106,0.844806,0.518300,1.559297e-07,0.000395
9,0.66,0.518958,1.913185e-08,0.000138,0.839896,0.518652,2.726004e-07,0.000522


In [9]:
columns = list(df_approximate)
columns.remove("variance")
columns.remove("block_var")

df_approximate[columns].to_csv(
    "../data/tables/1D1N_approximate.dat",
    index=False,
    sep="&",
    header=[
        r"$\alpha$", r"$\langle E_L\rangle$", r"$\sigma$",
        r"$A$", r"$t_C$", r"$\sigma_b$"],
    float_format="%.5f",#["%.2f", "%.5f", "%.5f", "%.5f", "%.5f", "%.5f"],
    line_terminator="\\\\ \n",
    quotechar=" ")#,
    #quoting=csv.QUOTE_NONE)

In [10]:
columns = list(df_analytical)
columns.remove("variance")
columns.remove("block_var")

df_analytical[columns].to_csv(
    "../data/tables/1D1N_analytical.dat",
    index=False,
    sep="&",
    header=[
        r"$\alpha$", r"$\langle E_L\rangle$", r"$\sigma$",
        r"$A$", r"$t_C$", r"$\sigma_b$"],
    float_format="%.5f",#["%.2f", "%.5f", "%.5f", "%.5f", "%.5f", "%.5f"],
    line_terminator="\\\\ \n",
    quotechar=" ")#,
    #quoting=csv.QUOTE_NONE)